# Neural Network Basics

In this tutorial we will implement a sample neural network for audio processing. You will learn how to load data, build data loaders and networks, and how to train and evaluate a network. Learning both the fundamental concepts and advanced applications of neural networks become necessary and rewarding nowadays.

You can easily find tremendous amount of tutorials on neural networks - both the platforms/libraries and the basic concepts. You can also easily find a lot of opensource projects for all the fancy models proposed and published in recent years. Here we will focus on a simple pipeline for a speech processing project by building an autoencoder for a given input signal. I hope that by going through this pipeline you can get a clear picture of what you need to do in training and evaluation of a neural network.

In general a machine learning project consists of 3 stages: **data preparation**, **model definition**, and **training and evaluation functions**. We will go through each of them. Here we use [**Pytorch**](https://pytorch.org/) for all the models since Pytorch is one of the most widely-used deep learning platforms nowadays and is especially easy to learn for beginners.


Note that in this course we **do not** assume that you have a GPU for model training - the data size and model size are selected so that you can finish all the homeworks even with your laptop's CPU. So don't worry if your laptop is not that powerful!

In [1]:
# we first import all the libraries we intend to use
import numpy as np
import librosa
import os
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## 1. Data Preparation

Here I will show my way to do data preparation and dataset generation, but there are definitely many other ways to do that 

I'll start with the raw waveforms - suppose we have a folder that contains many waveforms, and we want to load them, process them and save them into a dataset for easy loading during training/testing. The audio I/O is already covered in the tutorial for signal processing with Python, and here I won't describe that again.

There are several libraries that I use the most. Despite for the standard libraries and audio I/O library *librosa* and *soundfile*, the library I typically use to generate (small-scale) dataset is [*h5py*](https://www.h5py.org/), a Python wrapper for the *hdf5* format files. HDF is the abbreviation of Hierarchical Data Format, a binary data format for storing datasets.

Given the path of a directory, we can scan through it, find all the .wav files, and save their paths:

In [2]:
import h5py

dir_path = '../wav_data'  # directory path

# walk through the directory, find the files with .wav extension
wav_files = []
for (dirpath, dirnames, filenames) in os.walk(dir_path):
    for file in filenames:
        if '.wav' in file:
            wav_files.append(dirpath+'/'+file)
        
num_data = len(wav_files)
print('Number of .wav files: {:2d}'.format(num_data))
print('Example path: ' + wav_files[0])

Number of .wav files: 319
Example path: ../wav_data/sx449.wav


Here we simply use 319 utterances as an example. Once we have the path to the waveforms, we can load each of them, calculate any feature we want (e.g. spectrogram), and save them to a dataset. Here we will save the raw waveforms as well as the log-power spectrograms.

Note that *hdf5* format assumes that each item in the dataset has a fixed shape. For 1-D feature vectors, all features should have the same length, and for 2-D feature matrices all features should have both the same width and height. For waveforms with different length, we need to either truncate them or zero-pad them to the same length. For simplicity here we set the maximum length to 2 seconds, and truncate or pad them accordingly.

We split the 319 files into a training set, a validation set and a test set. I'll use 200 for training, 50 for validation and 69 for testing.

In [4]:
# create blank datasets with h5py

tr_name = 'tr_set.hdf5'
val_name = 'val_set.hdf5'
test_name = 'test_set.hdf5'

tr_dataset = h5py.File(tr_name, 'a')
val_dataset = h5py.File(val_name, 'a')
test_dataset = h5py.File(test_name, 'a')

# maximum length of waveforms 
sr = 16000  # sample rate
length_wave_max = 2 * sr

# STFT window and hop size
n_fft = 512
n_hop = n_fft // 4

for i in range(num_data):
    # first 200 files for training, next 50 for validation, and last 69 for testing
    
    # load the wavefiles
    y ,_ = librosa.load(wav_files[i],sr=sr)  # the default sample rate for them is 16kHz, but you can also change that
    
    # truncate or zero-pad the signal
    y = y[:length_wave_max]
    if len(y) < length_wave_max:
        y = np.concatenate([y, np.zeros(length_wave_max-len(y))])
    
    # calculate log-power spectrogram in decibel scale
    spec = librosa.stft(y, n_fft=n_fft, hop_length=n_hop)
    # the 1e-8 here is added for numerical stability, in case that log10(0) happens
    log_power_spec = 10*np.log10(np.abs(spec)**2 + 1e-8)  # shape: (n_fft/2+1, T)
    num_frame = log_power_spec.shape[1]  # this should be the same for all the utterances, since we map them into same length
    
    # save them into the hdf5 file
    
    if i < 200:
        if i == 0:
            # create sub-datasets
            tr_dataset.create_dataset('waveform', shape=(200, length_wave_max), dtype=np.float32)
            tr_dataset.create_dataset('spec', shape=(200, n_fft//2+1, num_frame), dtype=np.float32)
        
        tr_dataset['waveform'][i] = y
        tr_dataset['spec'][i] = log_power_spec
    elif i < 250:
        if i == 200:
            # create sub-datasets
            val_dataset.create_dataset('waveform', shape=(50, length_wave_max), dtype=np.float32)
            val_dataset.create_dataset('spec', shape=(50, n_fft//2+1, num_frame), dtype=np.float32)
        
        val_dataset['waveform'][i-200] = y
        val_dataset['spec'][i-200] = log_power_spec
    else:
        if i == 250:
            # create sub-datasets
            test_dataset.create_dataset('waveform', shape=(69, length_wave_max), dtype=np.float32)
            test_dataset.create_dataset('spec', shape=(69, n_fft//2+1, num_frame), dtype=np.float32)
        
        test_dataset['waveform'][i-250] = y
        test_dataset['spec'][i-250] = log_power_spec
    
tr_dataset.close()
val_dataset.close()
test_dataset.close()

In most of the cases we need to do some normalization or whitening on the training set. Here we normalize the log-power spectrogram across the frames, so that each frame in the entire dataset should be zero mean and unit variance. This is often called the mean-variance normalization (MVN) operation. Note that we calculate the mean and variance statistics only using the training set, and apply them to both validation set and testing set.

In certain cases the normalization is done within the neural networks. We will probably get into it in future homeworks.

In [ ]:
# load the training set
tr_dataset = h5py.File(tr_name, 'a')
tr_spec = tr_dataset['spec']  # shape: (num_data, n_fft/2+1, T)
tr_spec = np.transpose(tr_spec[:], (0, 2, 1)).reshape(-1, n_fft//2+1)  # shape: (num_data*T, n_fft/2+1)
tr_mean = np.mean(tr_spec, axis=0)  # shape: (n_fft/2+1)
tr_var = np.var(tr_spec, axis=0)  # shape: (n_fft/2+1)
tr_std = np.sqrt(tr_var + 1e-8)  # again for numerical stability

# apply normalization to all the datasets

val_dataset = h5py.File(val_name, 'a')
test_dataset = h5py.File(test_name, 'a')

tr_dataset['spec'][:] = (tr_dataset['spec'][:] - tr_mean.reshape(1,-1,1)) / tr_std.reshape(1,-1,1)
val_dataset['spec'][:] = (val_dataset['spec'][:] - tr_mean.reshape(1,-1,1)) / tr_std.reshape(1,-1,1)
test_dataset['spec'][:] = (test_dataset['spec'][:] - tr_mean.reshape(1,-1,1)) / tr_std.reshape(1,-1,1)

tr_dataset.close()
val_dataset.close()
test_dataset.close()

# save the mean and std information in files
np.save('training_mean', tr_mean)
np.save('training_std', tr_std)

Now we have the datasets generated and ready for use. In Pytorch, we can define *DataLoader* classes to load our saved datasets into Pytorch-friendly formats that can be used by the models we will define in the next session.

In [ ]:
from torch.utils.data import Dataset, DataLoader

batch_size = 8

# a class to load the saved h5py dataset
class dataset_pipeline(Dataset):
    def __init__(self, path):
        super(dataset_pipeline, self).__init__()

        self.h5pyLoader = h5py.File(path, 'r')
        
        self.spec = self.h5pyLoader['spec']
        
        self._len = self.spec.shape[0]  # number of utterances
    
    def __getitem__(self, index):
        spec_item = torch.from_numpy(self.spec[index].astype(np.float32))
            
        return spec_item
    
    def __len__(self):
        return self._len
    
# define data loaders
train_loader = DataLoader(dataset_pipeline('tr_set.hdf5'), 
                          batch_size=batch_size, 
                          shuffle=True,  # this ensures that the sequential order of the training samples will be shuffled for different training epochs
                         )

validation_loader = DataLoader(dataset_pipeline('val_set.hdf5'), 
                               batch_size=batch_size, 
                               shuffle=False,  # typically we fix the sequential order of the validation samples
                              )

dataset_len = len(train_loader)
log_step = dataset_len // 4

## 2. Model Definition

We will go through the implementation of 3 types of basic layers here - fully-connected layer (FC layer), recurrent layer, and convolutional layer. After that we will show how to construct a deeper network, and you will be asked to implement one network by yourself.

### 2.1 FC Layers

An FC layer simply contains a weight matrix together with a bias vector, and performs a linear transformation on an input feature followed by a nonlinear activation function. *help(nn.Linear)* provides a detailed description about the weights and the operation of an FC layer. (always use *help()* when you don't know what happens inside a function!)

In [ ]:
input = torch.randn((10, 100))  # create a random input features of shape (batch_size, feature_dimension)
FC_layer = nn.Linear(100, 20)  # An FC layer
FC_activation = nn.ReLU()  # nonlinear activation function
output = FC_activation(FC_layer(input))  # process the input
print("Shape of input:", tuple(input.data.shape))
print("Shape of output:", tuple(output.data.shape))

### 2.2 Recurrent Layers

Recurrent layers are used for sequence modeling. Time steps in the input sequence is sent to the recurrent layer in a sequential order. A basic recurrent layer not only contains a weight matrix and a bias vector, but also maintains a * hidden state* that store the information processed so far. In other words, the output at the current time step not only depends on the current input, but also depends on the historical input at all previous steps (summarized in the current hidden state).

In [ ]:
input = torch.randn((2, 10, 100))  # create a batch of random sequential features with shape (batch_size, sequence_length, feature_dimension)
RNN_layer = nn.RNN(100, 20, num_layers=1, nonlinearity='tanh', batch_first=True)  # A basic RNN layer
init_hidden = torch.zeros(1, 2, 20)  # initial hidden state is typically zero matrices
output, output_hidden = RNN_layer(input, init_hidden)  # process the input
# you can also set init_hidden to None, and the layer with automatically treat it as zero matrices.
#output, output_hidden = RNN_layer(input, None)  # the output will be the same as above

print("Shape of input:", tuple(input.data.shape))
print("Shape of output:", tuple(output.data.shape))

Beyond the basic RNN layer, we typically use more advanced RNN layers with a memory cell. One of the most famous layer is the long-short term memory (LSTM) layer.

In [ ]:
LSTM_layer = nn.LSTM(100, 20, num_layers=1, batch_first=True)  # A basic RNN layer
init_hidden = torch.zeros(1, 2, 20)  # initial hidden state is typically zero matrices
init_cell = torch.zeros(1, 2, 20)  # now you also have a initial cell state
output, (output_hidden, output_cell) = LSTM_layer(input, (init_hidden, init_cell))  # process the input
# similarly, init_hidden and init_cell can both be None
# output, (output_hidden, output_cell) = LSTM_layer(input, (None, None))  # same as above
print("Shape of input:", tuple(input.data.shape))
print("Shape of output:", tuple(output.data.shape))

### 2.3 Convolutional Layers

Convolutional layers are the most popular building blocks for computer vision tasks, but we can also use them in speech processing networks. In image processing people typically use 2-D convolutional blocks as images are 2-D, but in audio processing we can either use 1-D convolution or 2-D convolution, depending on our input feature.

If we want to directly manipulate the input waveforms, we can use 1-D convolutional layers (remember that Fourier transform can also be defined by the convolution between the signal and the sinusoid basis functions!). If we want to use spectrograms as input feature, we can use either 1-D or 2-D convolutional layers, where 1-D convolution treats each frame as an entire feature and performs sequence modeling, and 2-D convolution treats the spectrogram as an image.

Kernel size, stride size and padding size are extremely important in convolutional layers. In speech and audio processing you often want to have the same output size as the input, and in this case you need to be very careful about the stride size and padding size as they need to be manually calculated from your kernel size.

In [ ]:
input = torch.randn((2, 1, 160))  # create a batch of random input waveforms with shape (batch_size, in_channels, num_samples)
Conv1d_layer = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=0, stride=1)  # A basic Conv1d layer
Conv1d_activation = nn.ReLU()  # nonlinera activation
output = Conv1d_activation(Conv1d_layer(input))  # process the input

print("Shape of input:", tuple(input.data.shape))
print("Shape of output:", tuple(output.data.shape))

# play with different configurations of padding size and stride size
Conv1d_layer = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1, stride=1)
output = Conv1d_activation(Conv1d_layer(input))  # process the input

print("Shape of input:", tuple(input.data.shape))
print("Shape of output:", tuple(output.data.shape))

Conv1d_layer = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1, stride=2)
output = Conv1d_activation(Conv1d_layer(input))  # process the input

print("Shape of input:", tuple(input.data.shape))
print("Shape of output:", tuple(output.data.shape))

In [ ]:
input = torch.randn((2, 1, 33, 40))  # create a batch of random input spectrograms with shape (batch_size, in_channels, frequency_dim, time_step)
Conv2d_layer = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=(1, 1), stride=(1, 1))  # A basic Conv1d layer
Conv2d_activation = nn.ReLU()  # nonlinera activation
output = Conv2d_activation(Conv2d_layer(input))  # process the input

print("Shape of input:", tuple(input.data.shape))
print("Shape of output:", tuple(output.data.shape))

## Construct a Deeper Model

Now we know how to build single FC/RNN/CNN layers. It's time for us to build a deeper network using those basic layers. Let's start with a multilayer perceptron (MLP) built from FC layers.


In [ ]:
# a 3 layer MLP
class MLP(nn.Module):
    def __init__(self, feature_dim):
        super(MLP, self).__init__()
        
        # layers, and activation function
        
        self.feature_dim = feature_dim
        self.hidden_unit = 128  # number of hidden units
        
        # input layer
        # nn.Sequential allows you to wrap multiple layers into a single large layer
        # all the layers inside an nn.Sequential object will run in their sequential order
        self.layer1 = nn.Sequential(nn.Linear(self.feature_dim, self.hidden_unit),
                                    nn.Sigmoid()
                                   )
        
        # do the same for two other layers
        # hidden layer
        self.layer2 = nn.Sequential(nn.Linear(self.hidden_unit, self.hidden_unit),
                                    nn.Sigmoid()
                                   )
        
        # output layer
        # here we do not apply a nonlinear activation function as we want to do autoencoding
        self.layer3 = nn.Linear(self.hidden_unit, self.feature_dim)
        
    # the function for the forward pass of network (i.e. from input to output)
    def forward(self, input):
        # the input is a batch of spectrograms with shape (batch_size, frequency_dim, time_step)
        # we want the MLP to be applied to the frequency_dim dimension, i.e. process each frame independently
        # we need to rotate the last two dimensions and reshape it into a 2-D matrix with shape (batch_size*time_step, frequency_dim)
        
        batch_size, freq_dim, time_step = input.shape
        
        input = input.transpose(1, 2).contiguous()  # (batch, time, freq), swap the two dimensions
        input = input.view(batch_size*time_step, freq_dim)  # (batch*time, freq), reshaping
        
        # pass it through the three layers
        output = self.layer1(input)  # (batch*time, hidden_unit)
        output = self.layer2(output)  # (batch*time, hidden_unit)
        output = self.layer3(output)  # (batch*time, freq)
        
        # reshape back
        output = output.view(batch_size, time_step, freq_dim)  # (batch, time, freq)
        output = output.transpose(1, 2).contiguous()  # (batch, freq, time), swap the two dimension back
        
        return output

In [ ]:
# test if the model works well
model_MLP = MLP(feature_dim=257)

input = torch.randn((2, 257, 40))
output = model_MLP(input)

print("Shape of input:", tuple(input.data.shape))
print("Shape of output:", tuple(output.data.shape))

## Objective and Optimizer

Now we have the model and the data prepared. We need to define an optimizer that allows us the train the model, and an objective function that the optimizer uses to measure whether the model is generating correct outputs and update the model weights based on the values of the objective functions. This is done by backpropagation - I believe you all know this.

A *learning rate* needs to be defined for an optimizer - it determines how much the model parameters are adjusted each time it receives a new batch of training data. There are many discussions on the selection of learning rates - large learning rates can prevent the successful convergence of the model, and small learning rates can make the convergence speed way too slow. Here we typically set the initial learning rate to 0.01 in an empiricaly way.

In [ ]:
# define the optimizer
# typically I use Adam, but you can use others
optimizer = optim.Adam(model_MLP.parameters(), lr=1e-3)

Now let's define the objective function. One of the most widely-used objective function is the mean-square error (MSE) function that compares the Euclidean distance between the input and output:

In [ ]:
def MSE(input, output):
    # input shape: (batch, freq, time)
    # output shape: (batch, freq, time)
    
    batch_size = input.shape[0]
    input = input.view(batch_size, -1)
    output = output.view(batch_size, -1)
    loss = (input - output).pow(2).mean(1)  # (batch_size, 1)
    return loss.mean()

## Training and Testing

Finally we can define the functions for training and validation. For the task of autoencoding, the input and the output of the model are the same.

In [ ]:
# function for training and validation

def train(model, epoch, versatile=True):
    start_time = time.time()
    model = model.train()  # set the model to training mode. Always do this before you start training!
    train_loss = 0.
    
    # load batch data
    for batch_idx, data in enumerate(train_loader):
        batch_spec = data
        
        # clean up the gradients in the optimizer
        # this should be called for each batch
        optimizer.zero_grad()
        
        output = model(batch_spec)
        
        # MSE as objective
        loss = MSE(batch_spec, output)
        
        # automatically calculate the backward pass
        loss.backward()
        # perform the actual backpropagation
        optimizer.step()
        
        train_loss += loss.data.item()
        
        # OPTIONAL: you can print the training progress 
        if versatile:
            if (batch_idx+1) % log_step == 0:
                elapsed = time.time() - start_time
                print('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.2f} | MSE {:5.4f} |'.format(
                    epoch, batch_idx+1, len(train_loader),
                    elapsed * 1000 / (batch_idx+1), 
                    train_loss / (batch_idx+1)
                    ))
    
    train_loss /= (batch_idx+1)
    print('-' * 99)
    print('    | end of training epoch {:3d} | time: {:5.2f}s | MSE {:5.4f} |'.format(
            epoch, (time.time() - start_time), train_loss))
    
    return train_loss
        
def validate(model, epoch):
    start_time = time.time()
    model = model.eval()  # set the model to evaluation mode. Always do this during validation or test phase!
    validation_loss = 0.
    
    # load batch data
    for batch_idx, data in enumerate(validation_loader):
        batch_spec = data
        
        # you don't need to calculate the backward pass and the gradients during validation
        # so you can call torch.no_grad() to only calculate the forward pass to save time and memory
        with torch.no_grad():
        
            output = model(batch_spec)
        
            # MSE as objective
            loss = MSE(batch_spec, output)
        
            validation_loss += loss.data.item()
    
    validation_loss /= (batch_idx+1)
    print('    | end of validation epoch {:3d} | time: {:5.2f}s | MSE {:5.4f} |'.format(
            epoch, (time.time() - start_time), validation_loss))
    print('-' * 99)
    
    return validation_loss

Now let's define some hyperparameters and train the model.

In [ ]:
# hyperparameters

total_epoch = 100  # train the model for 100 epochs
model_save = 'best_model_MLP.pt'  # path to save the best validation model

# main function

training_loss = []
validation_loss = []

for epoch in range(1, total_epoch + 1):
    training_loss.append(train(model_MLP, epoch))
    validation_loss.append(validate(model_MLP, epoch))
    
    if training_loss[-1] == np.min(training_loss):
        print('      Best training model found.')
    if validation_loss[-1] == np.min(validation_loss):
        # save current best model on validation set
        with open(model_save, 'wb') as f:
            torch.save(model_MLP.state_dict(), f)
            print('      Best validation model found and saved.')
    
    print('-' * 99)

Test phase is the same as the validation phase, and I skip the details here.

## Exercise

It's your turn to play with the models - implement a 2-layer LSTM network. The template is provided.

In [ ]:
# a 2-layer LSTM
class DeepLSTM(nn.Module):
    def __init__(self, feature_dim):
        super(DeepLSTM, self).__init__()
        
        # layers, and activation function
        
        self.feature_dim = feature_dim
        self.hidden_unit = 128  # number of hidden units
        
        # TODO: first LSTM layer
        self.LSTM1 = nn.LSTM(self.feature_dim, self.hidden_unit, num_layers=1, batch_first=True)
        
        # TODO: second LSTM layer
        self.LSTM2 = nn.LSTM(self.hidden_unit, self.hidden_unit, num_layers=1, batch_first=True)
        
        # you can also jointly define a 2-layer LSTM 
        # self.deep_LSTM = nn.LSTM(self.feature_dim, self.hidden_unit, num_layers=2, batch_first=True)
        
        # output layer
        # this is an FC layer 
        self.output = nn.Linear(self.hidden_unit, self.feature_dim)
        
    # the function for the forward pass of network (i.e. from input to output)
    def forward(self, input):
        # the input is a batch of spectrograms with shape (batch_size, frequency_dim, time_step)
        
        batch_size, freq_dim, time_step = input.shape
        
        # note that LSTM layers require input with shape (batch_size, sequence_length, feature_dim)
        # so here we need to reshape the input
        
        input = input.transpose(1, 2).contiguous()  # (batch, time, freq), swap the two dimensions
        
        # TODO: pass it through the 2 LSTM layers
        
        output, output_hidden = self.LSTM1(input, None)
        output, output_hidden = self.LSTM2(output, None)
        
        # the output should have shape (batch, time, hidden_unit)
        # pass to the output layer
        
        output = self.output(output.contiguous().view(batch_size*time_step, -1))  # *batch*time, freq
        
        # reshape back
        output = output.view(batch_size, time_step, freq_dim)  # (batch, time, freq)
        output = output.transpose(1, 2).contiguous()  # (batch, freq, time), swap the two dimension back
        
        return output
    
model_LSTM = DeepLSTM(feature_dim=257)
optimizer = optim.Adam(model_LSTM.parameters(), lr=1e-3)

In [ ]:
# train the model you implemented

total_epoch = 100  # train the model for 100 epochs
model_save = 'best_model_LSTM.pt'  # path to save the best validation model

# main function

training_loss = []
validation_loss = []

for epoch in range(1, total_epoch + 1):
    training_loss.append(train(model_LSTM, epoch))
    validation_loss.append(validate(model_LSTM, epoch))
    
    if training_loss[-1] == np.min(training_loss):
        print('      Best training model found.')
    if validation_loss[-1] == np.min(validation_loss):
        # save current best model on validation set
        with open(model_save, 'wb') as f:
            torch.save(model_LSTM.state_dict(), f)
            print('      Best validation model found and saved.')
    
    print('-' * 99)

Take a look at the training speed and training and validation loss, compare with the 3-layer MLP. What do you find?

In [ ]:
# TODO: enter your discussion here

## Load a saved model weight

Loading a saved model weight from a file (like the one above) is also simple.

In [ ]:
# load the best validation model for the MLP
model_MLP.load_state_dict(torch.load('best_model_MLP.pt'))

## Debugging a Network

Debugging a neural network is not the same as debugging other codes, since there can be various reasons your model doesn't work or you have NaNs in your outputs. Here I'll cover several common issues and a few simple way to debug.

The most straightforward way for debugging is to print some of the intermdeidate outputs or the network parameters. But you can also print the gradient of an intermediate output to see if it's in good range. To do this, you can either *register a hook* on that variable and ask it to print out its gradient, or directly call *.grad* to extract the gradient. Note that once you registered a hook, all receiving gradients will be printed out, but calling *.grad* will only print the gradient once.

In [ ]:
# a simple computational graph
a = nn.Parameter(torch.empty(10).uniform_(-1, 1))
b = nn.Parameter(torch.empty(10).uniform_(-1, 1))
c = (a * b).sum()
# to print the gradient of a during backpropagation, register a 'print' hook on it
a.register_hook(print)
c.backward()
# and let's check if that is the true gradient
# the gradient received by a equals to b
torch.allclose(b, a.grad)

Sometimes you meet NaNs or inf in the gradient, and by printing it you may have a chance to find where are them. There could be many reasons for NaN/inf, and three most common ones are **gradient explosion**, **divide-by-zero**, and **gradient not defined**.

In [ ]:
# gradient explosion
a = nn.Parameter(torch.empty(10).uniform_(0, 1e2))
b = (torch.pow(a, 10)).prod()
# to print the gradient of a during backpropagation, register a 'print' hook on it
a.register_hook(print)
b.backward()

Here inf occurs when the gradient is too large. This often happens when you have power function. A simple way to avoid it is to use log when you apply power operations:

In [ ]:
# avoid gradient explosion by log
log_a = torch.log(a)
log_b = 50*log_a.sum()
b = torch.exp(log_b)
log_b.backward()  # use log_b to backpropagate instead of b

Divide-by-zero can occur in many places. I recommend you always add a small scalar (e.g. 1e-8) to the denominator whenever you have a division operation.

In [ ]:
# divide-by-zero
b = (a / 0).sum()
b.backward()

Gradient not defined is a special case for some functions at some value. For example, if you manually calculate the power of L2-norm at 0, you will have NaN as gradient; but if you call *norm* function, you will have 0. This is related to the definition of **sub-gradient**, and I won't go to the details about it.

In [ ]:
d = nn.Parameter(torch.zeros(1))
d.register_hook(print)
e = d.pow(2).sqrt()
e.backward()
e = d.norm(p=2)
e.backward()

Again, I suggest you to always be careful to fractional exponents and check if a variable contains 0. There's actually a built-in function in Pytorch that helps you detect these issues:

In [ ]:
# automatically detect these issues
from torch.autograd import detect_anomaly

with detect_anomaly():
    d = nn.Parameter(torch.zeros(1))
    d.register_hook(print)
    e = d.pow(2).sqrt()
    e.backward()

Note that this will only detect NaN but not inf, and using *detect_anomaly()* will significantly slow down your training. Nevertheless, it is pretty useful when you try to find out what happened to your model.